In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5 pyspark-shell'

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import *
from pyspark.sql.functions import * 

spark = SparkSession \
    .builder \
    .appName("FHIRstream") \
    .getOrCreate()

In [2]:
# Subscribe to a pattern
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("subscribePattern", "fhir") \
  .load()
df = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [3]:
# schema = StructType([StructField("resourceType", StringType()), StructField("name", StringType())])
# df = df.select(from_json(df["value"], schema))

In [4]:
df.writeStream \
  .option("checkpointLocation", "/home/jovyan") \
  .option("es.nodes", "es") \
  .format("es") \
  .start("fhir")